# Les arbres de régression et de classification

**<span style='color:blue'> Objectifs de la séquence</span>** 
* Comprendre&nbsp;:
    * Le principe d'un arbre de décision,
    * La construction d'un arbre de décision.
* Savoir interpréter un arbre de décision.
* Manipuler&nbsp;:
    * Les arbres de décision via la librairie $\texttt{sklearn}$.



 ----

## I. Introduction

Les arbres de décision forment une catégorie de modèles clés en *marchine learning*. Bien que n'étant pas les modèles les plus performants par eux-mêmes, ils forment la brique de base des forêts aléatoires, souvent sur le podium des modèles les plus performants. De plus, leur facilité d'interprétation leur offre un atout non négligeable.

Les [arbres de décision](https://fr.wikipedia.org/wiki/Arbre_de_d%C3%A9cision) s’appuient sur une structure de données qu’on appelle un [arbre](https://fr.wikipedia.org/wiki/Arbre_(th%C3%A9orie_des_graphes)) (thanks captain obvious). Le principe d'un arbre de décision est illustré par la figure suivante&nbsp;:


![Decision tree](https://raw.githubusercontent.com/maximiliense/lmiprp/main/Travaux%20Pratiques/Machine%20Learning/Introduction/data/Introduction/cart.jpg)

Imaginons l'étudiant John Smith caractérisé par une note en licence en informatique de 14, une note en mathématiques de 15 et qui écoute en cours. Sa classification se fera comme suit&nbsp;:
1.  On regarde la racine&nbsp;: l'étudiant écoute en cours, on prend donc la branche de droite,
2.  On regarde sa note en math&nbsp;: elle est supérieure à 10, on en conclut qu'il obtiendra son module de *machine learning*.

L'idée est qu'à chaque nœud notre arbre va découper le sous-espace dont "il s'occupe par un hyperplan de manière à définir deux sous-espaces. L'un sera traité par les nœuds de la branche droite et l'autre par ceux de la branche gauche. Créons maintenant un jeu de données synthétique illustrant le scénario précédent.

In [ ]:
import numpy as np

def create_student_dataset(n):
    # la premiere colonne est la note d'informatique,
    # la seconde la note de math
    X = np.random.uniform(0, 20, size=(n, 2))
    conditions = np.stack([
        X[:, 0] > 11,
        X[:, 1] > 8
    ]).T
    y = np.any(
        np.stack(
         [np.all(conditions[:, 0:2], axis=1), X[:, 1] > 18]
        ), 
        axis=0)
    return X, y
    
X, y = create_student_dataset(45)

In [ ]:
import matplotlib.pyplot as plt

def plot_student(X, y, tree=None):
    plt.figure(figsize=(12, 8))
    plt.scatter(X[y, 0], X[y, 1], color='yellow', edgecolors='black', 
                label='Réussi le module de machine learning')
    plt.scatter(X[(1-y).astype(bool), 0], X[(1-y).astype(bool), 1], 
                c='purple', edgecolors='black', 
                label='Échoue le module de machine learning')
    plt.xlabel('Informatique')
    plt.ylabel('Mathématiques')
    plt.legend()
    plt.show()

plot_student(X, y)

Construisons avec le *framework* $\texttt{sklearn}$ quelques arbres de profondeurs différentes sur ce problème et observons le découpage de l'espace qui en résulte.

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt


fig_x_min, fig_x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
fig_y_min, fig_y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(
    np.arange(fig_x_min, fig_x_max, 0.1),
    np.arange(fig_y_min, fig_y_max, 0.1)
)

plt.figure(figsize=(15, 5))
for i in range(1, 4):
    ax = plt.subplot(1, 3, i)
    tree = DecisionTreeClassifier(max_depth=i)
    tree.fit(X, y)
    Z = tree.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, alpha=0.4)
    plt.scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor='k')
    plt.xlabel('Informatique')
    plt.ylabel('Mathématiques')
    plt.title('Arbre de décision avec une profondeur de ' + str(i))
plt.show()



On voit très bien à chaque étape supplémentaire le fait qu'un hyperplan sépare l'espace en deux (ou qu'on s'arrête). Visualison maintenant l'arbre du graphique de droite ainsi construit.

In [ ]:
plt.figure(figsize=(12, 8))
_ = plot_tree(tree, rounded=True, fontsize=14)

La première ligne de chaque nœud représente le critère qui nous fera aller à droite ou à gauche. La seconde ligne "gini" indique la condition qui a permis à l'algorithme de décider de créer de nouveaux nœuds ou de s'arrêter (i.e. lorsque gini vaut $0$ ou que la profondeur est atteinte, on s'arrête). *Samples* est le nombre de points de notre jeu de données qui se retrouvent concernés par le nœud et enfin, *value* est la prédiction. Il s'agit ici d'un problème de classification binaire, et la première case représente le label $0$ et la seconde le label $1$. Étant donné un nœud feuille, la prédiction à faire est le vote à la majorité simple. On peut également demander une estimation des probabilités et le modèle retournera le vecteur suivant&nbsp;:

```python
value / value.sum()
```

**<span style='color:blue'> Critère d'arrêt</span>** Vous remarquerez peut-être selon le tirage du jeu de données que l'arbre de profondeur 2 est le même que l'arbre de profondeur 3. C'est normal, nous minimisons déjà l'erreur et il n'est pas utile d'aller plus loin. Cela est réprésenté par une valeur *gini* qui vaut 0 dans le schéma de l'arbre.


 ----

## II. Construction d'un arbre de classification

La racine de notre arbre regroupe tous les points de notre jeu de données. Comment décider de quelle manière nous devons ensuite créer de nouveaux nœuds voire même si nous devons en créer ? À la première itération, l'idée va être de considérer toutes les valeurs possibles de seuil $t$ et des variables $j$ qui sépareraient notre jeu de données $S$ en deux groupes $S^{(l)}$ et $S^{(r)}$ tels que $\forall x\in S_n^{(l)}\Rightarrow x_j \leq t$ et $\forall x\in S_n^{(r)}\Rightarrow x_j > t$. Le choix du seuil se fera sur un critère qu'on appelle "le gain d'information"&nbsp;:


$$\mathcal{IG}(S^{l}, S^{r})=\mathcal{I}(S)-\frac{n_l}{n}\mathcal{I}(S^{l})-\frac{n_r}{n}\mathcal{I}(S^{r}),$$

Où $n_r=|S^{(r)}|$ et $n_l=|S^{(l)}|$ et $\mathcal{I}$ est une "mesure d'impureté" d'un groupe que nous définirons plus tard. L'idée est de mesurer à quel point notre seuil $t$ et le choix de la variable $j$ ont bien séparé notre jeu de données en deux groupes où les classes sont moins mélangées que dans le jeu complet $S$.

Plusieurs métriques d'impureté existent dont nous listons quelques exemples&nbsp;:

* Impureté de Gini
* Entropie
* Erreur de classification

L'impureté de Gini se calcule de la manière suivante&nbsp;:

$$\mathcal{I}_{\mathcal{G}}(S)=\sum_k p_k(1-p_k),$$

où $p_i$ indique la proportion d'éléments de la classe $i$ dans l'ensemble $S$. L'entropie est donnée par la formule $\mathcal{I}_\mathcal{E}(S)=-\sum_k p_k \text{log}_2 p_k$ et l'erreur de classification par $\mathcal{I}_{EC}(S)=1-\text{max}_k(p_k)$. On remarque que dans les figures de l'exemple ci-dessus, nous avons utilisé l'impureté de Gini ! 

L'étape précédente est ensuite répétée pour chaque sous-groupe. Plusieurs stratégies d'arrêt sont possibles. Tout d'abord, lorsque chaque feuille ne contient qu'un seul élément et ne peut plus être divisée en deux. Ou lorsque le *split* n'améliore pas l'*accuracy* d'un gain relatif d'au moins $\alpha$ (hyperparamètre qu'on fixe). Une autre stratégie consiste à construire l'arbre complet et à ensuite éliminer les branches qui ne satisfont pas certains critères, par exemple au travers d’une validation croisée. Et bien sûr, lorsque l'arbre a atteint la profondeur maximum fixée par l'utilisateur.

## III. Construction d'un arbre de régression
Dans le cas d'un arbre de régression, le critère de *split* est différent du cas précédent. Nous considérons cette fois-ci comme critère d'erreur (l'impureté précédente) le *mean-squared error* ou MSE&nbsp;:


$$MSE = \sum_i(\bar{y} – y_i)^2/n.$$

Le split est choisi minimisant la quantité suivante&nbsp;: 

$$\frac{n_l}{n}MSE(S^{(l)})+\frac{n_r}{n}MSE(S^{(r)}).$$

Ce n'est rien d'autre que la moyenne pondérée des erreurs.

## IV. Applications

**<span style='color:blue'> Exercice</span>** 
**Pour les différents jeux de données suivants, proposez un type d'arbre de classification (régression ou classification). Quel critère de split a été utilisé ? Quelle est la profondeur de l’arbre calculé ? Quelle règle de décision (i.e. le chemin de décision dans l'arbre) a été utilisée pour le premier élément du jeu de test (uniquement les jeux de données 1 avec max_depth=3 et 2) ?**


 ----

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

**Jeu de données 1**

In [ ]:
data = datasets.fetch_california_housing()

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.5, shuffle=True
)

In [ ]:
####### Complete this part ######## or die ####################
...
print('Le score R2:', model.score(X_test, y_test))
...
print('On prédit:', model.predict(X_test[:1]), 'pour', X_test[:1])
###############################################################


**Jeu de données 2**

In [ ]:
data = datasets.load_iris()

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.5, shuffle=True
)

In [ ]:
####### Complete this part ######## or die ####################
...
print('L\'accuracy de notre modèle:', model.score(X_test, y_test))
...
print('On prédit:', model.predict(X_test[:1]), 'pour', X_test[:1])
###############################################################


**Jeu de données 3**

In [ ]:
data = datasets.fetch_covtype()

X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.5, shuffle=True
)

In [ ]:
####### Complete this part ######## or die ####################
......

print('L\'accuracy de notre modèle:', model.score(X_test, y_test))

print('On prédit:', model.predict(X_test[:1]), 'pour', X_test[:1])
###############################################################
